# Gruppering

In [3]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

In [4]:
# Find alle Excel-filer i en mappe
filer = glob.glob("/Users/mathias/Desktop/Uni/4. Semester/Dataprojekt/Rådata/2023 AAR vaskeri data/PLC*.xlsx")

# Indlæs alle filer og saml dem i en liste
dataframes = [pd.read_excel(fil, skiprows=2) for fil in filer]

# Eventuelt kombiner alle til én DataFrame
samlet_df = pd.concat(dataframes, ignore_index=True)

In [5]:
skjorte_data = samlet_df[samlet_df['Produkt - Produkt'].str.contains('skjorte', case=False, na=False)]

søgeord_buks = ['buks', 'benkl', 'benklæder', 'unisexben', 'benk']
pattern_buks = '|'.join(søgeord_buks)
buks_data = samlet_df[samlet_df['Produkt - Produkt'].str.contains(pattern_buks, case=False, na=False)]

søgeord_tshirt = ['t-shirt', 't shirt', 'tshirt']
pattern_tshirt = '|'.join(søgeord_tshirt)
tshirt_data = samlet_df[samlet_df['Produkt - Produkt'].str.contains(pattern_tshirt, case=False, na=False)]

søgeord_sweatshirt = ['sweatshirt', 'sweat shirt', 'sweat', 'fleece']
pattern_sweatshirt = '|'.join(søgeord_sweatshirt)
sweatshirt_data = samlet_df[samlet_df['Produkt - Produkt'].str.contains(pattern_sweatshirt, case=False, na=False)]

søgeord_jakke = ['jakke', 'vest']
pattern_jakke = '|'.join(søgeord_jakke)
jakke_data = samlet_df[samlet_df['Produkt - Produkt'].str.contains(pattern_jakke, case=False, na=False)]

forklæde_data = samlet_df[samlet_df['Produkt - Produkt'].str.contains('forklæde', case=False, na=False)]

søgeord_sok = ['sok', 'strømpe']
pattern_sok = '|'.join(søgeord_sok)
sok_data = samlet_df[samlet_df['Produkt - Produkt'].str.contains(pattern_sok, case=False, na=False)]

kittel_søgeord = ['kittel', 'kit.', 'kit']
pattern_kittel = '|'.join(kittel_søgeord)
kittel_data = samlet_df[samlet_df['Produkt - Produkt'].str.contains(pattern_kittel, case=False, na=False)]


# udskrift samlet data minus skjorte data og buks data
print((f'Før filtrering: {len(samlet_df)} rækker, efter filtrering: {(len(skjorte_data) + len(buks_data) + len(tshirt_data) + len(sweatshirt_data) + len(jakke_data) + len(forklæde_data) + len(sok_data) + len(kittel_data))} rækker, i procent: {((len(skjorte_data) + len(buks_data) + len(tshirt_data) + len(sweatshirt_data) + len(jakke_data) + len(forklæde_data) + len(sok_data) + len(kittel_data)) / len(samlet_df)) * 100:.2f}%'))

Før filtrering: 248711 rækker, efter filtrering: 219345 rækker, i procent: 88.19%


In [6]:
# Brug mere specifik matching med word boundaries ellerrækkefølge
import re

# Metode 1: Prioriteret matching (bedste metode)
def kategoriser_produkt(produktnavn):
    produktnavn = str(produktnavn).lower()
    
    # Tjek i prioriteret rækkefølge (mest specifik først)
    if 'forklæde' in produktnavn:
        return 'Forklæde'
    elif any(ord in produktnavn for ord in ['sweat']):
        return 'Sweatshirt'
    elif any(ord in produktnavn for ord in ['t-shirt', 'polo', 'tshirt']):
        return 'T-shirt'
    elif 'kittel' in produktnavn or re.search(r'\bkit[\s\.]', produktnavn):
        return 'Kittel'
    elif any(ord in produktnavn for ord in ['skjorte', 'skj.']):
        return 'Skjorte'
    elif any(ord in produktnavn for ord in ['jakke', 'vest', 'pilotjakke']):
        return 'Jakke'
    elif any(ord in produktnavn for ord in ['buks', 'bukser', 'benk', 'benklæder', 'unisexben', 'jeans']):
        return 'Bukser'
    elif 'sok' in produktnavn or 'strømpe' in produktnavn:
        return 'Sokker'
    elif 'overall' in produktnavn or 'kedeldr' in produktnavn or 'heldragt' in produktnavn:
        return 'Overall'
    elif 'busseron' in produktnavn:
        return 'Busseron'
    else:
        return 'Andet'

# Tilføj kategori
samlet_df['Kategori'] = samlet_df['Produkt - Produkt'].apply(kategoriser_produkt)

# Nu kan du filtrere uden overlap
skjorte_data = samlet_df[samlet_df['Kategori'] == 'Skjorte']
buks_data = samlet_df[samlet_df['Kategori'] == 'Bukser']
tshirt_data = samlet_df[samlet_df['Kategori'] == 'T-shirt']
sweatshirt_data = samlet_df[samlet_df['Kategori'] == 'Sweatshirt']
jakke_data = samlet_df[samlet_df['Kategori'] == 'Jakke']
forklæde_data = samlet_df[samlet_df['Kategori'] == 'Forklæde']
sok_data = samlet_df[samlet_df['Kategori'] == 'Sokker']
kittel_data = samlet_df[samlet_df['Kategori'] == 'Kittel']
busseron_data = samlet_df[samlet_df['Kategori'] == 'Busseron']
andre_data = samlet_df[samlet_df['Kategori'] == 'Andet']

# Tjek resultater
print(samlet_df['Kategori'].value_counts())


# skriv andre data til csv
andre_data.to_csv('andre_data.csv', index=False)
tshirt_data.to_csv('tshirt_data.csv', index=False)
sweatshirt_data.to_csv('sweatshirt_data.csv', index=False)
kittel_data.to_csv('kittel_data.csv', index=False)
buks_data.to_csv('buks_data.csv', index=False)



# print alle unikke produktnavne
print(samlet_df['Produkt - Produkt'].unique())

# skriv alle unikke buks data produktnavne til csv for
unikke_buks = pd.DataFrame(buks_data['Produkt - Produkt'].unique(), columns=['Produktnavn'])
unikke_buks.to_csv('unikke_buks.csv', index=False)



# skriv alle unikke produktnavne for andre til csv
unikke_andre = pd.DataFrame(andre_data['Produkt - Produkt'].unique(), columns=['Produktnavn'])
unikke_andre.to_csv('unikke_andre.csv', index=False)


# skriv 

print(len(unikke_andre))

Kategori
Bukser        77905
T-shirt       75396
Jakke         23423
Andet         22146
Skjorte       18147
Overall       11763
Sweatshirt     8503
Busseron       5735
Kittel         2868
Forklæde       2095
Sokker          730
Name: count, dtype: int64
<StringArray>
[           '04290 Skjorte Marine P/B',        '01085 T-Shirt Gråmelange p/b',
   '07963 Busseron 1/Indv L/Æ Hvid Pb', '02788 Sweatshirt Mblå Elis Chauffør',
    '04231 Benkl m knæ kblå p/b ICON1',  '04574 Overall m knæ marine bom ICO',
               '00215 Termojakke Hvid',        '01717 T-shirt unisex kblå pb',
     '14152 Benkl m/lom hvid low risk',   '01083 T-Shirt unisex hvid k/æ P/B',
 ...
   '09442 Forklæde Strib Kort Sort Gr',  '22458 Knickers Altona grå/sort p/b',
        '00262 Shorts Hr Grå/Sort P/B',   '13713 Håndværkershorts Zafra sort',
    '22077 Regnbuks Kl1 Wolfsberg Gul',    '22301 Shorts Revi EN20471 kl2 or',
 '22828 Kokkejakke so CHEF L/Æ T/K CC',   '26331 Fleecejakke Sheffield or/ma',
      '28869 Se

In [7]:
# Tjek at ingen produkter er i flere kategorier (skulle være umuligt nu)
total = samlet_df['Kategori'].value_counts().sum()
unikke = len(samlet_df)
print(f"Total kategoriserede: {total}")
print(f"Unikke rækker: {unikke}")
print(f"Overlap: {total - unikke}")  # Skal være 0!

Total kategoriserede: 248711
Unikke rækker: 248711
Overlap: 0
